Import Google map Library

In [ ]:
!pip install google_streetview
!pip install googlemaps

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Import Python Libraries

In [ ]:
import google_streetview.api
import googlemaps


import requests
import os

import pandas as pd
from pandas import DataFrame

import matplotlib.pyplot as plt #For creating histograms

import seaborn

import cv2
import numpy as np

Define street view image funtion to download images

In [ ]:
def StreetViewImage(obj_id, Lat,Long,OutputDir):

    #Get location

    Location = str(Lat) + "," + str(Long)

    #Get address

    API_KEY = ''

    gm = googlemaps.Client(key=API_KEY)

    Address = gm.geocode(Location)[0]["formatted_address"]

    Address = Address.replace('  ', ' ')
    Address = Address.replace(",","")

    # Define parameters for street view api
    params = [{
        'size': '640x640', # max 640x640 pixels
        'location': Address,
        'pitch': 15,
        'FOV': 15,
        'key': API_KEY
    }]

    # Create a results object
    results = google_streetview.api.results(params)


    #Create specific sub-folder within directory

    # SubFolder = OutputDir + "/" + "_" + Address
    SubFolder = OutputDir
    image_folder = SubFolder + '/images/'
    meta_folder = SubFolder + '/metadata/'
    os.makedirs(image_folder, exist_ok=True)
    os.makedirs(meta_folder, exist_ok=True)
    if os.path.exists(SubFolder):

        # Define rename output convetion for Streetview images and metadata

        Img_Orig = SubFolder + "/" + "gsv_0.jpg"
        Meta_Orig = SubFolder + "/" + "metadata.json"

        Img_Updated = image_folder + f"{obj_id}_{Location}_{Address}" + ".jpg"
        Meta_Updated = meta_folder + f"{obj_id}_{Location}_{Address}" + ".json"

        # Download images to directory 'downloads'
        try:
            results.download_links(SubFolder)
            os.rename(Img_Orig, Img_Updated)
            os.rename(Meta_Orig, Meta_Updated)

            # Crop image
            # Get image name
            img = cv2.imread(Img_Updated)
            cv2.imshow("original", img)
            # Cropping an image
            cropped_image_centre = img[160:480,215:425]
            # Display cropped image
            cv2.imshow("cropped", cropped_image_centre)
            # Save the cropped image
            cv2.imwrite(image_folder + f"{obj_id}_{Location}_{Address}" + "_centre.jpg", cropped_image_centre)
        except:
            pass
    else:
        print("File already exists: " + Address)

Define and read input file

In [ ]:
input_file = r'/content/F4_Surface.csv'

df = pd.read_csv(input_file)

df.head(3)

Define output location

In [ ]:
output_dir = r"/content/drive/MyDrive/F4_Surface"

Extract images

In [ ]:
for ind in df.index:

#   Get parameters
    Lat,Long,output_dir = df['CNTRD_LAT'][ind], df['CNTRD_LONG'][ind],output_dir
    obj_id = df['OBJECTID *'][ind]
    print(obj_id, Lat, Long, output_dir)


#   Extract function
    StreetViewImage(obj_id, Lat,Long,output_dir)

End